In [117]:
import pandas as pd
import numpy as np
from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Participant codes

YH0040 : 18-34ans, Femme, Droitière

YH0038 : 18-34ans, Femme, droitière

YH0033 : 18-34ans, Femme, droitière

OH0042 : 35-59ans, Homme, Gaucher, amputé (donc amputé du bras droit, prothèse cosmétique)

YH0031 : 18-34ans, Homme, droitier

OH0037 : 35-59 ans, Homme, Ambidextre

OH0034 : 35-59ans, Homme, Gaucher, amputé (du bras droit, possède une prothèse myoélectrique)

OH0029 : 60-80ans, Femme, Droitière

OH0025 : 35-59ans, Femme, Droitière, amputé (du bras gauche, prothèse cosmétique)

# Primary features

In [14]:
ids = ['YH0040', 'YH0038', 'YH0033', 'OH0042', 'YH0031', 'OH0037', 'OH0034_1', 'OH0034_2', 'OH0029', 'OH0025_1', 'OH0025_2']
#ids = ['YH0040']

In [15]:
for id in ids:
    # Load data

    data_path = f'../data/ESK_data/body_kpts_{id}.npy'
    #data_npy_path = "../data/data_test_31.07.npy"
    data_npy = np.load(data_path)

    df_test = pd.DataFrame()
    df_test['time'] = np.array([time/30 for time in range(len(data_npy))])
    for i in range(17):
        df_point = pd.DataFrame(data_npy[:, i])
        df_test[f'point_{i}'] = df_point[[0, 1, 2]].to_numpy().tolist()
        df_test[f'point_{i}'] = [np.array(point) for point in df_test[f'point_{i}']]
    df_test.rename(columns = {'point_0': 'nose', 'point_1': 'lefteye',
                            'point_2': 'righteye', 'point_3': 'leftear', 'point_4': 'rightear',
                            'point_5':'leftshoulder', 'point_6':'rightshoulder', 
                            'point_8':'rightelbow', 'point_7':'leftelbow', 
                            'point_10':'rightwrist', 'point_9':'leftwrist',
                            'point_12':'righthip', 'point_11':'lefthip',
                            'point_13': 'leftknee', 'point_14': 'rightknee',
                            'point_15': 'leftfoot', 'point_16': 'rightfoot'}, inplace = True)

    """     joints = ['leftshoulder_flex', 'rightshoulder_flex', 'leftshoulder_abduc',
        'rightshoulder_abduc', 'leftelbow_flex', 'rightelbow_flex',
        'trunk_forward_flex', 'trunk_lateral_flex', 'trunk_rotation'] """
    df_test['neck'] = (df_test['leftshoulder'] + df_test['rightshoulder'])/2
    df_test['hips'] = (df_test['lefthip'] + df_test['righthip'])/2

    # Transform original coordinate to body coordinates

    new_df_test = transform_points(df_test)

    # Add joint angles

    new_df_test['leftshoulder_flex'] = angle_plane(new_df_test, 'leftelbow', 'leftshoulder', 'sagittal')
    new_df_test['rightshoulder_flex'] = angle_plane(new_df_test, 'rightelbow', 'rightshoulder', 'sagittal')
    new_df_test['leftshoulder_rot'] = angle_plane(new_df_test, 'leftelbow', 'leftshoulder', 'transverse')
    new_df_test['rightshoulder_rot'] = angle_plane(new_df_test, 'rightelbow', 'rightshoulder', 'transverse')

    add_naive_joint_angles(new_df_test, 'leftelbow', 'leftshoulder', 'lefthip')
    add_naive_joint_angles(new_df_test, 'rightelbow', 'rightshoulder', 'righthip')

    add_naive_joint_angles(new_df_test, 'leftwrist', 'leftelbow', 'leftshoulder')
    add_naive_joint_angles(new_df_test, 'rightwrist', 'rightelbow', 'rightshoulder')

    new_df_test['trunk_fwd_flex'] = angle_plane(df_test, trunk=True)

    new_df_test['trunk_lat_flex'] = angle_plane(new_df_test, trunk=True, plane = 'coronal')

    # Add centroids

    add_centroid(new_df_test, ['leftshoulder', 'leftelbow', 'leftwrist'], 'leftarm_centroid')
    add_centroid(new_df_test, ['rightshoulder', 'rightelbow', 'rightwrist'], 'rightarm_centroid')
    add_centroid(new_df_test, ['leftshoulder', 'leftelbow', 'leftwrist', 'rightshoulder', 'rightelbow', 'rightwrist'], 'arms_centroid')
    add_centroid(new_df_test, ['leftshoulder', 'rightshoulder', 'righthip', 'lefthip'], 'trunk_centroid')

    # Add velocities and acceleration (positions and angular)

    keys = ['nose', 'lefteye', 'righteye', 'leftear', 'rightear',
       'leftshoulder', 'rightshoulder', 'leftelbow', 'rightelbow',
       'leftwrist', 'rightwrist', 'lefthip', 'righthip', 'leftknee',
       'rightknee', 'leftfoot', 'rightfoot', 'neck', 'hips', 
       'leftarm_centroid', 'rightarm_centroid', 'arms_centroid', 'trunk_centroid']

    joints = ['leftshoulder_flex', 'rightshoulder_flex', 'leftshoulder_rot', 'rightshoulder_rot',
            'leftshoulder_angle', 'rightshoulder_angle', 'leftelbow_angle', 'rightelbow_angle', 
            'trunk_fwd_flex', 'trunk_lat_flex']

    add_velocities_acceleration(new_df_test, keys, joints)

    # Add the 3 components of body points/centroids separately
    add_each_components(new_df_test, keys)

    new_df_test.to_pickle(f'primary_features_{id}.pkl')

## Now secondary features based on events

### Events selection

0 : Reach for thing when both feet are on ground

1: Chop zucchini

2 : Chop eggplant

3 : Chop mushrooms 

4 : Chop peppers

5 : Chop tomatoes

6 : Add things to containers (pot, tup, plate)

7 : Stirring

8 : Add seasoning (not pepper)

9 : Washing hands (9)

n : ignore (10)

u : Doing dishes (11)

In [44]:
events = {'Reach to things':0, 'Chop zucchini':1, 'Chop eggplant':2, 'Chop mushrooms':3, 'Chop peppers':4, 'Chop tomatoes':5,
          'Transfer things to containers':6, 'Stirring':7, 'Seasoning':8, 'Washing hands':9, 'Doing dishes':11}

In [46]:
keys = ['nose', 'lefteye', 'righteye', 'leftear', 'rightear',
       'leftshoulder', 'rightshoulder', 'leftelbow', 'rightelbow',
       'leftwrist', 'rightwrist', 'lefthip', 'righthip', 'leftknee',
       'rightknee', 'leftfoot', 'rightfoot', 'neck', 'hips', 'time', 
       'leftarm_centroid', 'rightarm_centroid', 'arms_centroid', 'trunk_centroid']

for id in ids:
    df = pd.read_pickle(f'../data/ESK_data/primary_features_{id}.pkl')
    events_path = f'../data/ESK_data/{id}.npy'
    data_events = np.load(events_path)
    sec_features = pd.DataFrame()

    colnames = df.drop(labels = keys, axis = 1).columns.values

    for col in colnames:
        columns = []
        for key in events:
            idx = np.where(data_events == events[key])
            #Sometimes df and data_events do not have the same size
            if len(df) != len(data_events):
                idx = [id for id in idx[0] if id <= len(df)-1]
            sum_stats = df[[col]].iloc[idx].describe()
            sum_stats = sum_stats[col].to_dict()
            sum_stats['RoM'] = sum_stats['max'] - sum_stats['min']
            columns.append(sum_stats)
        sec_features[col] = columns
        
    indexes = {i: event for i, event in zip(range(len(events)), events.keys())}
    sec_features.index.name = 'Events'
    sec_features.rename(index = indexes, inplace = True)
    sec_features.to_pickle(f'secondary_features_{id}.pkl')